In [1]:
%pip install sparse

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import sparse
#from google.colab import drive
#drive.mount('/content/drive/')

In [3]:
#google driveのMy Drive内にあるinput.csvを選択
#df = pd.read_csv('/content/drive/My Drive/cpdecomposition/input/triple.csv')
df = pd.read_csv('./input/triple.csv')
df.head()

,subject,property,object
0,http://dbpedia.org/resource/Saint_Thomas_Health,http://dbpedia.org/ontology/regionServed,http://dbpedia.org/resource/Middle_Tennessee
1,http://dbpedia.org/resource/Saint_Thomas_Health,http://dbpedia.org/ontology/regionServed,http://dbpedia.org/resource/South_Central_Kent...
2,http://dbpedia.org/resource/Saint_Thomas_Health,http://dbpedia.org/ontology/locationCountry,http://dbpedia.org/resource/United_States
3,http://dbpedia.org/resource/Pangea3,http://xmlns.com/foaf/0.1/homepage,http://www.pangea3.com
4,http://dbpedia.org/resource/Pangea3,http://dbpedia.org/ontology/foundationPlace,http://dbpedia.org/resource/United_States


In [4]:
#Triples with some properties are removed
df = df[df['property'] != 'http://www.w3.org/2000/01/rdf-schema#seeAlso']
df = df[df['property'] != 'http://www.w3.org/2002/07/owl#differentFrom']
df = df[df['property'] != 'http://xmlns.com/foaf/0.1/depiction']
df = df[df['property'] != 'http://xmlns.com/foaf/0.1/homepage']

In [5]:
#Extracting resources
subjects = df['subject'].unique()
objects = df['object'].unique()
#Make a list of entities, properties
entities = np.insert(subjects, 0, objects)
entities = np.unique(entities)
properties =df['property'].unique()

In [6]:
#Create index conversion dictionaries for entities, properties
dict_ent = {}
dict_prop = {}

cnt = 0
for i in entities:
  dict_ent[i] = cnt
  cnt = cnt + 1

cnt = 0
for k in properties:
  dict_prop[k] = cnt 
  cnt = cnt + 1

In [7]:
#Create sparse.COO(coords, data, shape) parameterss
m1 = []
m2 = []
m3 = []
for triple in df.values:
  s, p, o = triple[0], triple[1], triple[2]
  m1.append(dict_ent[s])
  m2.append(dict_ent[o])
  m3.append(dict_prop[p])
coords = np.array([m1, m2, m3], dtype=np.float64)

data = 1.0

shape = [len(entities), len(entities), len(properties)]

In [8]:
print(coords)
print(data)
print(shape)
tensor = sparse.COO(coords, data, shape)

[[1.91839e+05 1.91839e+05 1.91839e+05 ... 1.91839e+05 1.91839e+05
  1.91839e+05]
 [1.44874e+05 2.03248e+05 2.25141e+05 ... 1.52508e+05 1.52508e+05
  9.21920e+04]
 [0.00000e+00 0.00000e+00 1.00000e+00 ... 2.00000e+00 3.00000e+00
  6.00000e+00]]
1.0
[247117, 247117, 97]


TypeError: only int indices permitted

In [10]:
tensor

NameError: name 'tensor' is not defined

In [9]:
%pip install tensorly

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [10]:
import tensorly as tl
#from tensorly.contrib.sparse import tensor, unfold
from tensorly.contrib.sparse.decomposition import parafac 
tl.get_backend()

'numpy'

In [11]:
import time

In [ ]:
start_time = time.time()

factors = parafac(tensor, rank=5)

end_time = time.time()
total_time = end_time - start_time
print('Took %d mins %d secs' % (divmod(total_time, 60)))